In [ ]:
#Penglab (Abuse of Google Colab for fun and profit)
#by mxrch

#Choose what you want to install
hashcat = True
john = False
hydra = False

#Which shell ? (only one, or none)
#⚠️ Don't select a shell to run bash commands in the Google Colab way (you'll see). Stable.
ssh = False #It uses ngrok. Stable and interactive, but take a little time to setup
python_shell = True #Useful for little hashes, but not for long hours, since you'll see the output only when it finish. Not interactive, but stable.
bash_shell = False #Useful for big hashes. but you'll don't see your input (surely a Google Colab protection). Stable.

#Wordlists (see weakpass.com to download them)
wordlists_dir = "wordlists"

rockyou = True #For CTFs, especially Hack The Box (133.44 Mb)
hashesorg2019 = False #Very heavy (12.79 Gb) but has a good rate, if you're determinated

#---------------------------

if (python_shell and bash_shell) or (bash_shell and ssh) or (ssh and python_shell) :
    print("Please do a choice")
    exit()

In [ ]:
#Wordlists
import os

os.system("wordlists_dir={}".format(wordlists_dir))
!mkdir ./$wordlists_dir
if rockyou:
    !printf "[+] Downloading the Rockyou wordlist...\n"
    !cd $wordlists_dir && wget https://download.weakpass.com/wordlists/90/rockyou.txt.gz
    !printf "[+] Wordlist downloaded !\n[+] Extraction...\n"
    !cd $wordlists_dir && gunzip rockyou.txt.gz
    !printf "[+] Finished !\n[+] Location : $(pwd)/$wordlists_dir/$(ls wordlists | grep rockyou)"

if hashesorg2019:
    !printf "[+] Downloading the HashesOrg2019 wordlist...\n"
    !cd $wordlists_dir && wget https://download.weakpass.com/wordlists/1851/hashesorg2019.gz
    !printf "[+] Wordlist downloaded !\n[+] Extraction...\n"
    !cd $wordlists_dir && gunzip hashesorg2019.gz
    !printf "[+] Finished !\n[+] Location : $(pwd)/$wordlists_dir/$(ls wordlists | grep hashesorg2019)"

[+] Downloading the Rockyou wordlist...
--2020-10-03 08:02:20--  https://download.weakpass.com/wordlists/90/rockyou.txt.gz
Resolving download.weakpass.com (download.weakpass.com)... 104.31.10.182, 104.31.11.182, 2606:4700:3038::681f:bb6, ...
Connecting to download.weakpass.com (download.weakpass.com)|104.31.10.182|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53357062 (51M) [application/octet-stream]
Saving to: ‘rockyou.txt.gz’

rockyou.txt.gz      100%[===================>]  50.88M  14.7MB/s    in 3.5s    

2020-10-03 08:02:24 (14.7 MB/s) - ‘rockyou.txt.gz’ saved [53357062/53357062]

[+] Wordlist downloaded !
[+] Extraction...
[+] Finished !
[+] Location : /content/wordlists/rockyou.txt

In [ ]:
#Install of hashcat
if hashcat:
    print("Installation of hashcat...")
    !apt install cmake build-essential -y
    !apt install checkinstall git -y
    !git clone https://github.com/hashcat/hashcat.git && cd hashcat && make -j 8 && make install

Installation of hashcat...
Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.17.1-1ubuntu0.7).
The following additional packages will be installed:
  file libmagic-mgc libmagic1
Suggested packages:
  gettext
The following NEW packages will be installed:
  checkinstall file libmagic-mgc libmagic1
0 upgraded, 4 newly installed, 0 to remove and 22 not upgraded.
Need to get 372 kB of archives.
After this operation, 5,735 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.4 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/mai

In [ ]:
#Install of john
if john:
    print("Installation of john...")
    !apt-get install john -y

In [ ]:
#Install of hydra
if hydra:
    print("Installation of hydra...")
    !apt install cmake build-essential -y
    !apt install checkinstall git -y
    !git clone https://github.com/vanhauser-thc/thc-hydra.git && cd thc-hydra && ./configure && make && make install

In [ ]:
#Setting up shells
from termcolor import colored
import os

if ssh:
    print("Setting up SSH...")
    !pip install git+https://github.com/demotomohiro/remocolab.git
    import remocolab
    remocolab.setupSSHD()

elif python_shell:
    os.system("touch /tmp/cmd && touch /tmp/status")

    template_cmd = "echo -n $(whoami)[$(hostname)[$(pwd) &> /tmp/status"
    os.system("bash -c '{}'".format(template_cmd))
    output = {"cmd": "", "status": ""}
    with open('/tmp/cmd', 'r') as f:
        output["cmd"] = f.read()
    with open('/tmp/status', 'r') as f:
        output["status"] = f.read()
    prefixes = output["status"].split("[")
    path = prefixes[2].replace('\n', '')
    prefix = colored(prefixes[0] + "@" + prefixes[1], "red") + ":" + colored(path, "cyan") + "$ "
    print("")

    while 1:
        print(prefix, end='')
        cmd = input()
        template_cmd = "cd {} && ".format(path) + "" + cmd + " &> /tmp/cmd ; echo $(whoami)[$(hostname)[$(pwd) &> /tmp/status"
        os.system("bash -c '{}'".format(template_cmd))
        output = {"cmd": "", "status": ""}
        with open('/tmp/cmd', 'r') as f:
            output["cmd"] = f.read()
        with open('/tmp/status', 'r') as f:
            output["status"] = f.read()
        prefixes = output["status"].split("[")
        path = prefixes[2].replace('\n', '')
        prefix = colored(prefixes[0] + "@" + prefixes[1], "red") + ":" + colored(path, "cyan") + "$ "
        print(output["cmd"])

elif bash_shell:
    !/bin/bash

else:
    print('\n🚨🚨🚨\n🎉 Your environment is ready !\nWant to run it from the code blocks ?')
    print('Just use the code block below and start your commands by a "!" (make sure to uncomment them).')
    print('▶️ When you\'re ready, hit the run button at the left of the code block !\n')
    print('|\n|\n|\nV')


root@87939bfd5707:/content$ ls -al
total 24
drwxr-xr-x  1 root root 4096 Oct  3 08:02 .
drwxr-xr-x  1 root root 4096 Oct  3 07:57 ..
drwxr-xr-x  1 root root 4096 Oct  1 16:28 .config
drwxr-xr-x 16 root root 4096 Oct  3 08:03 hashcat
drwxr-xr-x  1 root root 4096 Oct  1 16:28 sample_data
drwxr-xr-x  2 root root 4096 Oct  3 08:02 wordlists

root@87939bfd5707:/content$ whoami
root

root@87939bfd5707:/content$ ls
hashcat
sample_data
wordlists

root@87939bfd5707:/content$ cd hashcat

root@87939bfd5707:/content/hashcat$ ls
BUILD_CYGWIN.md
BUILD.md
BUILD_MSYS2.md
BUILD_WSL.md
charsets
deps
docs
example0.cmd
example0.hash
example0.sh
example400.cmd
example400.hash
example400.sh
example500.cmd
example500.hash
example500.sh
example.dict
extra
hashcat
hashcat.hcstat2
hashcat.hctune
include
layouts
Makefile
masks
modules
obj
OpenCL
README.md
rules
src
tools

root@87939bfd5707:/content/hashcat$ ./hashcat -h
hashcat (v6.1.1-110-g0db82afa0) starting...

Usage: hashcat [options]... hash|hashfile|hccap

In [ ]:
# Put your code here :
# !hashcat -h
# !john --help